# Flight delays and weather condition – Data Exploration

**Purpose**
- Explore data for flight delays and weather condition use cases
- Validate assumptions before adding API endpoints
- Prototype logic for FastAPI services

**Author:** Rashed  
**Date:** 2025-24-12


In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robikscube/flight-delay-dataset-20182022")

print("Path to dataset files:", path)

100%|██████████| 3.73G/3.73G [01:34<00:00, 42.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4


In [1]:
%ls /root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4

Airlines.csv                   Combined_Flights_2021.csv
Combined_Flights_2018.csv      Combined_Flights_2021.parquet
Combined_Flights_2018.parquet  Combined_Flights_2022.csv
Combined_Flights_2019.csv      Combined_Flights_2022.parquet
Combined_Flights_2019.parquet  raw/
Combined_Flights_2020.csv      readme.html
Combined_Flights_2020.parquet  readme.md


In [2]:
import pandas as pd
# We only need few columns for our analysis
KEEP_COLS = [
    # ---- Date & time (join keys) ----
    "FlightDate",
    "DepTimeBlk",

    # ---- Airports (for direction) ----
    "Origin",
    "Dest",

    # ---- Flight identity (API join) ----
    "IATA_Code_Operating_Airline",

    # ---- Targets / outcomes ----
    "DepDelayMinutes",

    # ---- Operational signal ----
    "Distance",
    "CRSElapsedTime",

    # ---- Cancellation info (to drop)----
    "Cancelled",

    # ---- Arrival time to calculate counts ----
    "ArrTimeBlk",

]

df_2018 = pd.read_parquet('/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4/Combined_Flights_2018.parquet', columns=KEEP_COLS)
df_2019 = pd.read_parquet('/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4/Combined_Flights_2019.parquet', columns=KEEP_COLS)
df_2020 = pd.read_parquet('/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4/Combined_Flights_2020.parquet', columns=KEEP_COLS)
df_2021 = pd.read_parquet('/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4/Combined_Flights_2021.parquet', columns=KEEP_COLS)
df_2022 = pd.read_parquet('/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4/Combined_Flights_2022.parquet', columns=KEEP_COLS)

Now we will need to concatenate the data from different years into a single DataFrame for easier analysis.

In [3]:
df = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022], ignore_index=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29193782 entries, 0 to 29193781
Data columns (total 10 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   FlightDate                   datetime64[us]
 1   DepTimeBlk                   object        
 2   Origin                       object        
 3   Dest                         object        
 4   IATA_Code_Operating_Airline  object        
 5   DepDelayMinutes              float64       
 6   Distance                     float64       
 7   CRSElapsedTime               float64       
 8   Cancelled                    bool          
 9   ArrTimeBlk                   object        
dtypes: bool(1), datetime64[us](1), float64(3), object(5)
memory usage: 2.0+ GB


In [4]:
df.describe()

,FlightDate,DepDelayMinutes,Distance,CRSElapsedTime,CRSArrTime
count,29193782,2.843070e+07,2.919378e+07,2.919376e+07,2.919378e+07
mean,2020-04-23 22:27:03.485606,1.278311e+01,7.797346e+02,1.387605e+02,1.489004e+03
min,2018-01-01 00:00:00,0.000000e+00,1.600000e+01,-2.920000e+02,1.000000e+00
25%,2019-03-18 00:00:00,0.000000e+00,3.540000e+02,8.800000e+01,1.108000e+03
50%,2020-02-08 00:00:00,0.000000e+00,6.260000e+02,1.210000e+02,1.515000e+03
75%,2021-07-17 00:00:00,5.000000e+00,1.014000e+03,1.690000e+02,1.915000e+03
max,2022-07-31 00:00:00,7.223000e+03,5.812000e+03,1.645000e+03,2.400000e+03
std,NaN,4.617337e+01,5.812739e+02,7.077316e+01,5.072880e+02


In [5]:
# TODO: do not train the model, until you make sure that you can obtain distance and CRSElapsedTime from the API
df.head()

,FlightDate,DepTimeBlk,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,Cancelled,CRSArrTime
0,2018-01-23,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1304
1,2018-01-24,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1304
2,2018-01-25,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1304
3,2018-01-26,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,False,1304
4,2018-01-27,1400-1459,ABY,ATL,9E,0.0,145.0,60.0,False,1500


Our goal is to predict delay, so we do not need cancelled flights. We will filter them out during the data loading phase.

In [3]:
df = df[df['Cancelled'] == 0].copy()
df.drop(columns=['Cancelled'], inplace=True)
df.head()

,FlightDate,DepTimeBlk,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,ArrTimeBlk
0,2018-01-23,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
1,2018-01-24,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
2,2018-01-25,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
3,2018-01-26,1200-1259,ABY,ATL,9E,0.0,145.0,62.0,1300-1359
4,2018-01-27,1400-1459,ABY,ATL,9E,0.0,145.0,60.0,1500-1559


Let's refine the date & time columns for departure and arrival times to create a proper datetime representation.

In [4]:
df['FlightDate'] = pd.to_datetime(df['FlightDate'])
df['Hour'] = df['DepTimeBlk'].str.slice(0, 2).astype(int)
df.drop(columns=['DepTimeBlk'], inplace=True)
df["datetime"] = (
    df["FlightDate"]
    + pd.to_timedelta(df["Hour"], unit="h")
)
df.head()

,FlightDate,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,Cancelled,ArrTimeBlk,Hour,datetime
0,2018-01-23,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359,12,2018-01-23 12:00:00
1,2018-01-24,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359,12,2018-01-24 12:00:00
2,2018-01-25,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359,12,2018-01-25 12:00:00
3,2018-01-26,ABY,ATL,9E,0.0,145.0,62.0,False,1300-1359,12,2018-01-26 12:00:00
4,2018-01-27,ABY,ATL,9E,0.0,145.0,60.0,False,1500-1559,14,2018-01-27 14:00:00


In [5]:
df['Hour_Arrival'] = df['ArrTimeBlk'].str.slice(0, 2).astype(int)
df['arrival_next_day'] = df['Hour_Arrival'] < df['Hour']
df["arr_datetime"] = (
    df["FlightDate"]
    + pd.to_timedelta(df["Hour_Arrival"], unit="h")
    + pd.to_timedelta(df["arrival_next_day"].astype(int), unit="D")
)
df.drop(columns=['ArrTimeBlk', 'Hour_Arrival', 'arrival_next_day', 'Hour', 'FlightDate'], inplace=True)


In [6]:
df.head()

,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,Cancelled,datetime,arr_datetime
0,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-23 12:00:00,2018-01-23 13:00:00
1,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-24 12:00:00,2018-01-24 13:00:00
2,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-25 12:00:00,2018-01-25 13:00:00
3,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-26 12:00:00,2018-01-26 13:00:00
4,ABY,ATL,9E,0.0,145.0,60.0,False,2018-01-27 14:00:00,2018-01-27 15:00:00


In order to calculate the number of flights departing from an airport, and arriving to the same airport, within the same hour, we need to group by the origin and deparuture datetime, and then do the same for arrival. Then we can add these counts as new features to the main dataframe.

First, we will group by 'Origin' and 'datetime' to get the count of departures per hour for each airport.

In [7]:
dep_counts = (
    df.groupby(["Origin", "datetime"])
      .size()
      .reset_index(name="departures_per_hour")
      .rename(columns={"Origin": "airport", "datetime": "hour"})
)

arr_counts = (
    df.groupby(["Dest", "arr_datetime"])
      .size()
      .reset_index(name="arrivals_per_hour")
      .rename(columns={"Dest": "airport", "arr_datetime": "hour"})
)


In [8]:
dep_counts.head()

,airport,hour,departures_per_hour
0,ABE,2018-01-01 06:00:00,2
1,ABE,2018-01-01 09:00:00,2
2,ABE,2018-01-01 17:00:00,1
3,ABE,2018-01-01 20:00:00,1
4,ABE,2018-01-02 06:00:00,3


In [9]:
arr_counts.head()

,airport,hour,arrivals_per_hour
0,ABE,2018-01-01 09:00:00,1
1,ABE,2018-01-01 16:00:00,1
2,ABE,2018-01-01 17:00:00,2
3,ABE,2018-01-01 19:00:00,1
4,ABE,2018-01-01 22:00:00,1


In [14]:
congestion = (
    dep_counts
    .merge(
        arr_counts,
        on=["airport", "hour"],
        how="outer"
    )
    .fillna(0)
)

congestion["scheduled_congestion"] = (
    congestion["departures_per_hour"]
    + congestion["arrivals_per_hour"]
)

congestion.head(10)

,airport,hour,departures_per_hour,arrivals_per_hour,scheduled_congestion
0,ABE,2018-01-01 06:00:00,2.0,0.0,2.0
1,ABE,2018-01-01 09:00:00,2.0,1.0,3.0
2,ABE,2018-01-01 16:00:00,0.0,1.0,1.0
3,ABE,2018-01-01 17:00:00,1.0,2.0,3.0
4,ABE,2018-01-01 19:00:00,0.0,1.0,1.0
5,ABE,2018-01-01 20:00:00,1.0,0.0,1.0
6,ABE,2018-01-01 22:00:00,0.0,1.0,1.0
7,ABE,2018-01-02 06:00:00,3.0,0.0,3.0
8,ABE,2018-01-02 09:00:00,1.0,1.0,2.0
9,ABE,2018-01-02 15:00:00,0.0,1.0,1.0


In [16]:
congestion[congestion['departures_per_hour'] == 113]

,airport,hour,departures_per_hour,arrivals_per_hour,scheduled_congestion
4473761,ORD,2020-11-21 18:00:00,113.0,19.0,132.0


Now let's merge these counts back into the main dataframe.

In [ ]:
df = df.merge(
    congestion[["airport", "hour", "scheduled_congestion"]],
    left_on=["Origin", "datetime"],
    right_on=["airport", "hour"],
    how="left",
    validate="many_to_one"
)
df

,Origin,Dest,IATA_Code_Operating_Airline,DepDelayMinutes,Distance,CRSElapsedTime,Cancelled,datetime,arr_datetime,airport,hour,departures_per_hour
0,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-23 12:00:00,2018-01-23 13:00:00,ABY,2018-01-23 12:00:00,1
1,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-24 12:00:00,2018-01-24 13:00:00,ABY,2018-01-24 12:00:00,1
2,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-25 12:00:00,2018-01-25 13:00:00,ABY,2018-01-25 12:00:00,1
3,ABY,ATL,9E,0.0,145.0,62.0,False,2018-01-26 12:00:00,2018-01-26 13:00:00,ABY,2018-01-26 12:00:00,1
4,ABY,ATL,9E,0.0,145.0,60.0,False,2018-01-27 14:00:00,2018-01-27 15:00:00,ABY,2018-01-27 14:00:00,1
